# Relations & Graph Traversal

**Use case:** A social network with users, posts, and follow relationships.

SurrealDB is both a document database and a graph database. This means you can
model relationships as *edges* between records, then traverse the graph with
simple queries.

This notebook covers:

- Creating relations between records with `relate()`
- Querying related records with `get_related()`
- Removing relations
- Eager loading with `fetch()` and `prefetch_related()`

## Prerequisites

A SurrealDB Docker container must be running. See `00_setup.ipynb` for details.

In [ ]:
# -- Setup -----------------------------------------------------------------
import os, sys
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))
from dotenv import load_dotenv
load_dotenv()

from src.surreal_orm import SurrealDBConnectionManager

SurrealDBConnectionManager.set_connection(
    os.getenv("SURREALDB_URL", "ws://localhost:8000"),
    os.getenv("SURREALDB_USER", "root"),
    os.getenv("SURREALDB_PASS", "root"),
    os.getenv("SURREALDB_NAMESPACE", "ns"),
    os.getenv("SURREALDB_DATABASE", "db"),
)
print("Connection configured.")

## Define Models

We define two models: `User` and `Post`. The relationship between them
("who wrote what" and "who follows whom") will be represented as graph
edges rather than foreign key columns.

In SurrealDB, a relation like `alice -> follows -> bob` creates a record
in the `follows` edge table. This is what makes SurrealDB a graph database.

In [ ]:
from src.surreal_orm import BaseSurrealModel, SurrealConfigDict


class User(BaseSurrealModel):
    """A user in the social network."""
    model_config = SurrealConfigDict(table_name="users")

    id: str | None = None
    name: str
    bio: str = ""


class Post(BaseSurrealModel):
    """A blog post authored by a user."""
    model_config = SurrealConfigDict(table_name="posts")

    id: str | None = None
    title: str
    content: str = ""
    # We store the author reference as a string field.
    # Relations (graph edges) are created separately.
    author_id: str | None = None


print("Models defined: User, Post")

## Create Users and Posts

Let's create a small social network: three users with a few blog posts.

In [ ]:
# Create users
alice = User(id="alice", name="Alice", bio="Python developer and blogger")
bob = User(id="bob", name="Bob", bio="JavaScript enthusiast")
charlie = User(id="charlie", name="Charlie", bio="DevOps engineer")

await alice.save()
await bob.save()
await charlie.save()
print(f"Created users: {alice.name}, {bob.name}, {charlie.name}")

# Create posts
post1 = Post(id="post1", title="Getting Started with SurrealDB", content="SurrealDB is a multi-model database...", author_id=f"users:{alice.id}")
post2 = Post(id="post2", title="Graph Databases Explained", content="Graph databases store data as nodes and edges...", author_id=f"users:{alice.id}")
post3 = Post(id="post3", title="JavaScript in 2026", content="The JS ecosystem continues to evolve...", author_id=f"users:{bob.id}")

await post1.save()
await post2.save()
await post3.save()
print(f"Created {3} posts.")

## Creating Relations -- `relate()`

The `relate()` method creates a graph edge between two records. The edge is
stored in its own table (e.g., `follows`, `wrote`).

```
alice  --follows-->  bob
alice  --follows-->  charlie
bob    --follows-->  alice
alice  --wrote-->    post1
alice  --wrote-->    post2
bob    --wrote-->    post3
```

This is the power of a graph database: relationships are first-class citizens,
not just foreign keys.

In [ ]:
# Alice follows Bob and Charlie
await alice.relate("follows", bob)
await alice.relate("follows", charlie)
print(f"{alice.name} now follows {bob.name} and {charlie.name}")

# Bob follows Alice back
await bob.relate("follows", alice)
print(f"{bob.name} now follows {alice.name}")

# Create "wrote" edges linking authors to their posts
await alice.relate("wrote", post1)
await alice.relate("wrote", post2)
await bob.relate("wrote", post3)
print("Created 'wrote' relations for all posts.")

## Querying Relations -- `get_related()`

Use `get_related()` to traverse the graph. The `direction` parameter controls
which way you traverse:

- **`"out"`** -- Follow outgoing edges (e.g., "who does Alice follow?")
- **`"in"`** -- Follow incoming edges (e.g., "who follows Alice?")

Under the hood, this generates SurrealQL graph traversal queries like
`SELECT VALUE out.* FROM follows WHERE in = users:alice`.

In [ ]:
# Who does Alice follow? (outgoing "follows" edges)
alice_follows = await alice.get_related("follows", direction="out")
print(f"{alice.name} follows:")
for record in alice_follows:
    print(f"  - {record}")

# Who follows Alice? (incoming "follows" edges)
alice_followers = await alice.get_related("follows", direction="in")
print(f"\nFollowers of {alice.name}:")
for record in alice_followers:
    print(f"  - {record}")

# What did Alice write? (outgoing "wrote" edges)
alice_posts = await alice.get_related("wrote", direction="out")
print(f"\nPosts by {alice.name}:")
for record in alice_posts:
    print(f"  - {record}")

## Removing Relations

You can remove individual edges with `remove_relation()`, or remove all
edges of a given type with `remove_all_relations()`.

This is useful for "unfollow" actions, cleanup before deletion, or resetting
a user's state.

In [ ]:
# Alice unfollows Charlie (remove a single relation)
await alice.remove_relation("follows", charlie)
print(f"{alice.name} unfollowed {charlie.name}")

# Verify: Alice should now only follow Bob
still_follows = await alice.get_related("follows", direction="out")
print(f"{alice.name} still follows: {still_follows}")

In [ ]:
# Remove ALL outgoing "follows" edges from Bob (bulk removal)
await bob.remove_all_relations("follows", direction="out")
print(f"Removed all of {bob.name}'s outgoing follows.")

# Verify: Bob follows nobody now
bob_follows = await bob.get_related("follows", direction="out")
print(f"{bob.name} follows: {bob_follows}")

In [ ]:
# You can also remove multiple relation types at once.
# This is handy before deleting a user -- clean up all their edges first.
await alice.remove_all_relations(["follows", "wrote"], direction="out")
print(f"Removed all outgoing 'follows' and 'wrote' edges from {alice.name}.")

## Eager Loading -- `fetch()`

When a field stores a record link (e.g., `author_id = "users:alice"`),
SurrealDB can resolve it inline with the `FETCH` clause. This avoids the
N+1 query problem -- instead of one query per post to get the author, you
get everything in a single round-trip.

The `fetch()` method maps directly to SurrealQL's `FETCH` clause.

In [ ]:
# Fetch posts and resolve the author_id record link inline
# This generates: SELECT * FROM posts FETCH author_id;
posts_with_authors = await Post.objects().fetch("author_id").exec()

print("Posts with fetched authors:")
for post in posts_with_authors:
    # When FETCH is used, author_id may be resolved to the full record
    print(f"  '{post.title}' -- author_id: {post.author_id}")

## Prefetch Related -- `prefetch_related()` and `Prefetch`

For more complex eager loading scenarios, `prefetch_related()` executes
a separate query per relation and attaches the results to each parent
instance. The `Prefetch` class gives you fine-grained control:

- **Custom queryset** -- Apply filters, ordering, or limits to the related query.
- **Custom attribute** -- Store results under a different attribute name.

This is useful when you want to load "the 5 most recent posts by each user"
rather than all posts.

In [ ]:
from src.surreal_orm import Prefetch

# Re-create the "wrote" relations for the demo
# (We removed Alice's earlier, so let's add them back)
await alice.relate("wrote", post1)
await alice.relate("wrote", post2)

# Prefetch with a custom queryset -- only posts whose title contains "SurrealDB"
users = await User.objects().prefetch_related(
    Prefetch(
        "wrote",
        queryset=Post.objects().filter(title__contains="SurrealDB"),
        to_attr="surreal_posts",
    ),
).exec()

print("Users with prefetched SurrealDB posts:")
for user in users:
    # The prefetched results are attached under to_attr
    surreal_posts = getattr(user, "surreal_posts", [])
    print(f"  {user.name}: {len(surreal_posts)} matching post(s)")
    for p in surreal_posts:
        print(f"    - {p.title}")

## Summary

| Operation                   | Method                                         | When to use                                 |
|-----------------------------|-------------------------------------------------|---------------------------------------------|
| Create edge                 | `a.relate("edge", b)`                          | Follow, like, wrote, owns, etc.             |
| Traverse outgoing           | `a.get_related("edge", direction="out")`        | "Who does A follow?"                        |
| Traverse incoming           | `a.get_related("edge", direction="in")`         | "Who follows A?"                            |
| Remove one edge             | `a.remove_relation("edge", b)`                 | Unfollow, unlike, etc.                      |
| Remove all edges            | `a.remove_all_relations("edge", direction=...)` | Cleanup before delete                       |
| Resolve record links inline | `QuerySet.fetch("field")`                      | Avoid N+1 for linked records                |
| Batch-load relations        | `QuerySet.prefetch_related(Prefetch(...))`     | Load relations with filters/custom attrs    |

## Cleanup

Remove all tables and edge tables created in this notebook.

In [ ]:
client = await SurrealDBConnectionManager.get_client()
await client.query("REMOVE TABLE users;")
await client.query("REMOVE TABLE posts;")
await client.query("REMOVE TABLE follows;")
await client.query("REMOVE TABLE wrote;")
print("Cleanup complete -- all tables removed.")